# Setup

In [32]:
import numpy as np
from agents_clean import Agent
from model_clean import Model
from hybrid_networks_clean import rewire_network, randomize_network_v2, create_hybrid_network
from tqdm import tqdm
import networkx as nx
import pandas as pd
import random
from simulations_functions_clean import generate_parameters_1, run_simulation_with_params, run_simulations
import numbers

# Generate Parameters

In [33]:
def generate_parameters_1(n,rewiring_type='hierarchical'):
    params_list = []
    for _ in range(n):
        n_agents = random.randint(10, 500)
        ba_degree = random.randint(1, int(n_agents/10))
        er_prob=random.uniform(0, 0.3)
        p_rewiring = random.uniform(0, 1)
        if rewiring_type=='hierarchical':
            G_er = nx.gnp_random_graph(n_agents, er_prob, directed=True)
            network = rewire_network(G_er, p_rewiring=p_rewiring)
        if rewiring_type=='randomize':
            G_ba = nx.barabasi_albert_graph(n_agents, ba_degree)
            network = randomize_network_v2(G_ba, p_rewiring=p_rewiring)
        uncertainty = random.uniform(0.001,0.01)
        n_experiments = random.randint(1, 1000)
        params = {
            "n_agents": n_agents,
            "ba_degree": ba_degree,
            "er_prob":er_prob,
            "p_rewiring": p_rewiring,
            "network": network,
            "uncertainty": uncertainty,
            "n_experiments": n_experiments,
            "rewiring_type":rewiring_type
        }
        params_list.append(params)
    return params_list

In [34]:
# Example usage
parameter_dictionaries = generate_parameters_1(2,rewiring_type='randomize')  # Generate parameter dictionaries
parameter_dictionaries

[{'n_agents': 449,
  'ba_degree': 28,
  'er_prob': 0.15509329321279014,
  'p_rewiring': 0.9759147509579581,
  'network': <networkx.classes.graph.Graph at 0x7fb4d55b1ab0>,
  'uncertainty': 0.0062259389242187635,
  'n_experiments': 713,
  'rewiring_type': 'randomize'},
 {'n_agents': 227,
  'ba_degree': 7,
  'er_prob': 0.04812164251943406,
  'p_rewiring': 0.7328902953049773,
  'network': <networkx.classes.graph.Graph at 0x7fb4d6e20370>,
  'uncertainty': 0.002638766837492727,
  'n_experiments': 348,
  'rewiring_type': 'randomize'}]

# Simulation Functions

In [35]:
def run_simulation_with_params(param_dict, number_of_steps=1000, show_bar=False,agent_type="beta"):
    # Extract the network directly since it's already a NetworkX graph object
    my_network = param_dict['network']
    # Other parameters are directly extracted from the dictionary
    my_model = Model(my_network, n_experiments=param_dict['n_experiments'], uncertainty=param_dict['uncertainty']
                    , agent_type=agent_type)
    # Run the simulation with predefined steps and show_bar option
    
    my_model.run_simulation(number_of_steps=number_of_steps, show_bar=show_bar)
  
    

    result_dict = {
        key: value 
        for key, value in param_dict.items() 
        if isinstance(value, (numbers.Number, str, tuple, list))}   
    result_dict['true_consensus'] = my_model.conclusion
    result_dict['true_consensus_alternative_stop'] = my_model.conclusion_alternative_stop
    df = pd.DataFrame(my_model.agents_choices) 
    result_dict['choice_consensus'] = df.mean()[len(df.mean())-1]
    result_dict['convergence_step'] = my_model.n_steps # takes note of the last reported step
    result_dict['agent_type']=agent_type
    
    return result_dict


def run_simulations(param_list, number_of_steps=1000, show_bar=False,agent_type="beta"):
    def run_simulation(param_dict):
        return run_simulation_with_params(param_dict, number_of_steps=number_of_steps, show_bar=show_bar,agent_type=agent_type)
    
    results = []
    # Iterate over each parameter dictionary in the list
    for params in tqdm(param_list, desc="Running simulations"):
        result = run_simulation(params)
        results.append(result)
    
    # Convert the list of results into a Pandas DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Run Simulations

In [36]:
parameter_dictionaries = generate_parameters_1(2,rewiring_type='randomize')  # Generate parameter dictionaries
print(parameter_dictionaries)
results_df = run_simulations(parameter_dictionaries, number_of_steps=1000, show_bar=False,agent_type="bayes")
results_df.head()

Running simulations:   0%|          | 0/2 [00:00<?, ?it/s]

Running simulations: 100%|██████████| 2/2 [01:30<00:00, 45.20s/it]

   n_agents  ba_degree   er_prob  p_rewiring  uncertainty  n_experiments  \
0       449         28  0.155093    0.975915     0.006226            713   
1       227          7  0.048122    0.732890     0.002639            348   

  rewiring_type  true_consensus  true_consensus_alternative_stop  \
0     randomize             1.0                            False   
1     randomize             1.0                            False   

   choice_consensus  convergence_step agent_type  
0               1.0              1000       beta  
1               1.0              1000       beta  


In [37]:
results_df.head()

,n_agents,ba_degree,er_prob,p_rewiring,uncertainty,n_experiments,rewiring_type,true_consensus,true_consensus_alternative_stop,choice_consensus,convergence_step,agent_type
0,449,28,0.155093,0.975915,0.006226,713,randomize,1.0,False,1.0,1000,beta
1,227,7,0.048122,0.732890,0.002639,348,randomize,1.0,False,1.0,1000,beta


In [ ]:
results_df.to_csv('results_df.csv', index=False)